-- Write a SQL statement to find out the names of research scholars (RS) who have not issued any books.  (Database:  LIS)

select students.student_fname from members inner join students on 
members.roll_no = students.roll_no where members.member_type = 'PG' and members.member_no not in 
(select book_issue.member_no from book_issue )



select student_fname from members inner join students on 
members.roll_no = students.roll_no  left join book_issue on members.member_no = book_issue.member_no 
where members.member_type = 'PG' and book_issue.doi is null

select * from members where member_type = 'PG'


SELECT student_fname, student_lname FROM students WHERE roll_no in 
(SELECT roll_no FROM members WHERE member_no IN
((SELECT member_no FROM members WHERE member_type = 'PG')EXCEPT(SELECT DISTINCT member_no FROM book_issue)))

 -- Write a SQL statement to find out the name of the oldest player from the team named“All Stars”.  (Database:  FLIS)

 select * from players inner join teams on teams.team_id = players.team_id  where teams.name = 'All Stars' and players.dob in

  (select min(players.dob) from players inner join teams on teams.team_id = players.team_id  where teams.name = 'All Stars')

  select players.name from players where dob =(select max(players.dob) from players inner join teams on 
players.team_id=teams.team_id where teams.name='All Stars')

SELECT players.name FROM players WHERE players.dob = 
(SELECT MIN(pl.dob)FROM players pl, teams te WHERE te.team_id = pl.team_id AND te.name = 'All Stars')

-- Write an SQL statement to find the unique book titles which are issued to “PG” students but not to “UG” students.(Database:  LIS)

select distinct title from book_catalogue where isbn_no in(

(select book_copies.isbn_no from book_copies where book_copies.accession_no in (
select  book_issue.accession_no from members inner join book_issue on book_issue.member_no = members.member_no 
where members.member_type = 'PG')) except 

(select book_copies.isbn_no from book_copies where book_copies.accession_no in (
select  book_issue.accession_no from members inner join book_issue on book_issue.member_no = members.member_no 
where members.member_type = 'UG')))


SELECT DISTINCT title FROM book_catalogue WHERE isbn_no IN (    SELECT isbn_no     FROM book_copies     WHERE accession_no IN (
        SELECT book_issue.accession_no         FROM members         INNER JOIN book_issue ON book_issue.member_no = members.member_no 
		        WHERE members.member_type = 'PG'    )     AND isbn_no NOT IN (        SELECT isbn_no         FROM book_copies 
				WHERE accession_no IN (            SELECT book_issue.accession_no             FROM members             
				INNER JOIN book_issue ON book_issue.member_no = members.member_no 
            WHERE members.member_type = 'UG'    )    ));

In [ ]:
import os 
import sys 

import psycopg2

database = sys.argv[1]	

user = os.environ.get('PGUSER') 

password = os.environ.get('PGPASSWORD') 

host = os.environ.get('PGHOST')

port = os.environ.get('PGPORT')

def is_prime(n):
    if n <= 1:
        return False
    if n == 2:
        return True
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

def connectDb(database = database, username= user, pwd = password, 
              address= host, portnum= port):
    conn = None
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect(database=database, user=username, 
                              password=pwd, host=address, port=portnum)
        curr = conn.cursor()
        
        curr.execute("select p.name , t.name , p.jersey_no FROM  players p left join  teams t  
        on t.team_id = p.team_id order by p.name , t.name  desc")
        data = curr.fetchall()
        prime_players = [(row[0], row[1]) for row in data if is_prime(row[2])]
      
         # Sort players by name and team in reverse alphabetical order
        prime_players.sort(key=lambda x: (x[0], x[1]), reverse=True)
      
        # Print the sorted list
        for player in prime_players:
            print(f"{player[0]}, {player[1]}")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        conn.close()
# close the connection
connectDb()